In [1]:
from pyiron_xyflow import ReactFlowWidget

In [2]:
import json

In [3]:
w = ReactFlowWidget()
w

ReactFlowWidget()

In [4]:
nodes = [
    {
      'id': '1',
      'data': { 'label': 'Hello' },
      'position': { 'x': 0, 'y': 0 },
      'type': 'input',
      'sourcePosition': 'right',
    },
    {
      'id': '2',
      'data': { 'label': 'message' },
      'position': { 'x': 200, 'y': 100 },
      'type': 'output',
      'targetPosition': 'left',      
    },
    {
      'id': '3',
      'data': { 'label': 'message' },
      'position': { 'x': 0, 'y': 100 },
      'type': 'customNode',
      'style': {
              'border': '1px black solid',
              'padding': 20
            },    
      'targetPosition': 'left',  
      'sourcePosition': 'right',  
    }
]

In [5]:
nodes_json = json.dumps(nodes)
w.send(nodes_json)